In [2]:
# !pip install pandas-ta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta


In [15]:
data = pd.read_csv("../data/22Y/22Y_tatasteel_1D.csv",index_col="date")
data

,open,high,low,close,volume
date,,,,,
2000-01-03 05:30:00+05:30,8.72,9.03,8.61,8.98,20025360.0
2000-01-04 05:30:00+05:30,8.84,9.01,8.43,8.88,15546330.0
2000-01-05 05:30:00+05:30,8.52,9.60,8.52,9.22,38390400.0
2000-01-06 05:30:00+05:30,9.36,9.96,9.36,9.91,25596190.0
2000-01-07 05:30:00+05:30,10.22,10.55,9.80,10.13,36405110.0
...,...,...,...,...,...
2022-10-10 05:30:00+05:30,102.00,103.50,101.15,103.20,36641275.0
2022-10-11 05:30:00+05:30,103.55,103.65,99.80,100.35,41083734.0
2022-10-12 05:30:00+05:30,100.35,101.20,99.00,100.55,41357469.0


In [17]:
# Adding indicators
data['RSI']=ta.rsi(data.close, length=15)
data['EMA_20']=ta.ema(data.close, length=20)
data['EMA_100']=ta.ema(data.close, length=100)
data['EMA_150']=ta.ema(data.close, length=150)
data['EMA_200']=ta.ema(data.close, length=200)
data['EMA_500']=ta.ema(data.close, length=500)


data['target'] = data['close']-data.open
data['target'] = data['target'].shift(-1)

data['trend'] = [1 if data.target[i]>0 else 0 for i in range(len(data))]

# Dropping null values after adding indicators and features
data.dropna(inplace=True)
data.reset_index(inplace = True)
# data.drop(['volume', 'date'], axis=1, inplace=True) # not required
data.drop([ 'date','target'], axis=1, inplace=True) # not required

In [5]:
data

,open,high,low,close,volume,RSI,EMA_20,EMA_100,EMA_150,EMA_200,EMA_500,trend
0,5.12,5.19,5.05,5.16,2053710.0,49.560493,5.189072,5.251564,5.522660,5.773724,6.756780,1
1,5.18,5.43,5.18,5.39,5428420.0,55.576843,5.208208,5.254306,5.520902,5.769906,6.751324,1
2,5.48,5.66,5.42,5.61,8511680.0,60.415705,5.246474,5.261349,5.522083,5.768315,6.746768,1
3,5.69,5.88,5.69,5.82,6797310.0,64.383464,5.301095,5.272412,5.526029,5.768829,6.743068,1
4,5.80,5.98,5.64,5.84,12243840.0,64.744066,5.352419,5.283651,5.530187,5.769538,6.739463,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5164,103.30,103.50,101.90,103.30,36080875.0,50.812590,102.356870,105.067605,107.191605,108.195521,99.624326,1
5165,102.00,103.50,101.15,103.20,36641275.0,50.608103,102.437168,105.030623,107.138736,108.145815,99.638600,0
5166,103.55,103.65,99.80,100.35,41083734.0,45.069673,102.238391,104.937937,107.048819,108.068245,99.641440,1
5167,100.35,101.20,99.00,100.55,41357469.0,45.517973,102.077591,104.851047,106.962742,107.993436,99.645067,0


In [18]:
# Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data)
print(data_set_scaled)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 4.73201197e-05
  2.00265845e-03 1.00000000e+00]
 [4.08663670e-04 1.61877782e-03 9.01963505e-04 ... 1.39189759e-05
  1.94403777e-03 1.00000000e+00]
 [2.45198202e-03 3.17010657e-03 2.56712690e-03 ... 0.00000000e+00
  1.89508683e-03 1.00000000e+00]
 ...
 [6.70412750e-01 6.64103602e-01 6.57392632e-01 ... 8.94918264e-01
  9.99941520e-01 1.00000000e+00]
 [6.48617355e-01 6.47578578e-01 6.51842087e-01 ... 8.94263841e-01
  9.99980488e-01 0.00000000e+00]
 [6.53725650e-01 6.51625523e-01 6.50454451e-01 ... 8.93576760e-01
  1.00000000e+00 0.00000000e+00]]


In [20]:
# creating X,Y sets
X = []
backcandles = 30
for j in range(data_set_scaled[0].size-1): # for each col
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]): # for each data set, store curr col value of last 30 (backcandles) days
        X[j].append(data_set_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
X=np.moveaxis(X, [0], [2])
print(X.shape)

# creating y and converting x to np array
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
# rows from 30th position to end cuz, we can't create training set for first 30 days, min 30 day required
# -1 because we are predicting trend which is last in cols

y=np.reshape(yi,(len(yi),1)) 
# converting 1d array to 2d ([1,2,3,4]--> [[1],[2],[3],[4]])
#  
print(X.shape)
print(y.shape)

(5139, 30, 11)
(5139, 30, 11)
(5139, 1)


In [21]:
# splitting data
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [23]:
# !pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import TimeDistributed

from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import History
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
import numpy as np
#tf.random.set_seed(20)
np.random.seed(10)


In [26]:
model = Sequential()
model.add(Input(shape=(backcandles, len(data_set_scaled[0])-1), name='lstm_input'))

model.add(LSTM(150, name='first_layer',activation="relu"))
model.add(Dense(1, name='dense_layer',activation="sigmoid"))

model.compile( loss = 'mean_squared_error',optimizer=optimizers.Adam())
model.fit(x=X_train, y=y_train)

113/113 [==============================] - 4s 28ms/step - loss: 0.2498


In [34]:
X_test.shape

(1542, 30, 11)

In [127]:
# lstm_input = Input(shape=(backcandles, len(data_set_scaled[0])), name='lstm_input')

# inputs = LSTM(150, name='first_layer',return_sequences=True)(lstm_input)
# inputs = LSTM(75, name='second_layer',return_sequences=True)(inputs)
# inputs = LSTM(50, name='third_layer',return_sequences=True)(inputs)
# inputs = Dense(1, name='dense_layer')(inputs)
# output = Activation('linear', name='output')(inputs)
# model = Model(inputs=lstm_input, outputs=output)
# adam = optimizers.Adam()
# # model.compile(optimizer=adam, loss='mse')
# model.compile(optimizer=adam, loss='mse')
# model.fit(x=X_train, y=y_train, batch_size=1, epochs=20, shuffle=True, validation_split = 0.1)

# # add lstm layer

In [27]:
y_pred = model.predict(X_test)
X_pred = model.predict(X_train)
# input size = X,30,12
# where, X = number of predictig examples
# 30 = look back days which is 30 in our case
# 12 = number of featues

# so we will be proving all 12 features of previuos 30 days. and each 30 day will be considered as 1 set


113/113 [==============================] - 1s 10ms/step


In [28]:
y_pred.shape

(1542, 1)

In [29]:
# y_pred_2 = []
# X_pred_2 = []
# for i in y_pred:
#     y_pred_2.append(i.mean())

# for j in X_pred:
#     X_pred_2.append(j.mean())

In [30]:
def classify(pred):
    for i in range(len(pred)):
        if pred[i]>=0.5:
            pred[i]=1
        else:
            pred[i]=0
# classify(y_pred_2)
# classify(X_pred_2)
classify(y_pred)
classify(X_pred)

In [31]:
from sklearn.metrics import accuracy_score
print("on training data - ",accuracy_score(X_pred,y_train))
print("on test data - ",accuracy_score(y_pred,y_test))
# increasing number of layers did increased the accurarcy to 0.54

on training data -  0.53628023352794
on test data -  0.5428015564202334


In [33]:
model.save('../models/lstm_trend_1.0')

INFO:tensorflow:Assets written to: ../models/lstm_trend_1.0\assets
